# Engineering Notebook: Design Exercise 2

### Start
- Getting started with gRPC—takes a bit to get used to, but the abstraction is nice.
- Once we get the hang of it, sending requests and getting responses is super easy.
- Built-in request-response handling makes life simpler.

- We want to reuse as much code as possible.
- Figuring out how to make gRPC work with our API-style actions takes some effort.
    - Feels a bit hacky, but it works well.
- Error handling is tricky—every request can fail.
    - We add an optional error field to every response.
    - Probably should've used gRPC’s built-in error handling instead. Oh well.

### Push notifications from server

- Need to figure out how to push notifications from the server.
    - gRPC’s request-stream response mode looks useful—lets us keep connections open.

- Alternative: Have clients spin up their own servers and let ours connect back.
    - Sounds too complicated

- First try: Use condition variables for thread safety.
    - Problem: Also want to avoid blocking the whole thread.
    - Okay this is actually pretty hard to do with gRPC

- Let's keep it simpler: ignore thread safety (sorry Eddie) and have threads block, I guess we're only supporting like 12 clients for now

### Evaluation of Message Size
- First problem: how do we see message sizes in gRPC?
    - Can use verbose logging, probably easiest

In [ ]:
import secrets
import grpc

from src.common import (
    protocol_pb2_grpc, 
    hash_password,
    Header,
    LoginRequest,
    MessageRequest,
)

TOKEN = ""
PASSWORD_1 = hash_password(secrets.token_bytes(16))
PASSWORD_2 = hash_password(secrets.token_bytes(16))

channel = grpc.insecure_channel("localhost:23456")
stub = protocol_pb2_grpc.BVChatStub(channel)
header = Header(login_token=b"")

def test_env_setup(stub: protocol_pb2_grpc.BVChatStub):
    # Create the necessary accounts
    request_1 = LoginRequest(username="user_1", password_hash=PASSWORD_1)
    request_1_size = len(request_1.SerializeToString())
    print("request 1 size", request_1_size)

    response_1 = stub.Register(request_1)
    response_1_size = len(response_1.SerializeToString())
    print("response 1 size", response_1_size)

    request_2 = LoginRequest(username="user_2", password_hash=PASSWORD_2)
    request_2_size = len(request_2.SerializeToString())
    print("request 2 size", request_2_size)

    response_2 = stub.Register(request_2)
    response_2_size = len(response_2.SerializeToString())
    print("response 2 size", response_2_size)


def test_fn_login(stub: protocol_pb2_grpc.BVChatStub):
    request = LoginRequest(username="user_1", password_hash=PASSWORD_1)
    request_size = len(request.SerializeToString())
    print("request size", request_size)

    response = stub.Login(request)
    response_size = len(response.SerializeToString())
    print("response size", response_size)

    return response.login_token

def test_fn_message(stub: protocol_pb2_grpc.BVChatStub):
    header = Header(login_token=TOKEN)
    request = MessageRequest(header=header, recipient="user_2", content=secrets.token_hex(64))
    request_size = len(request.SerializeToString())
    print("request size", request_size)


    response = stub.SendMessage(request)
    response_size = len(response.SerializeToString())
    print("response size", response_size)

In [ ]:
test_env_setup(stub)

In [ ]:
%%timeit
test_fn_login(stub)

In [ ]:
TOKEN = test_fn_login(stub)

In [ ]:
%%timeit
test_fn_message(stub)

### For comparison: our custom wire protocol

- Create account: sent 114 bytes, received 86 bytes.
- Login: sent 114 bytes, received 99 bytes. 7 runs, 1,000 loops each: 1.34ms ± 52.2 µs
- Send message: sent 241 bytes, received 238 bytes. 7 runs, 100 loops each: 2.23 ms ± 96.4 μs

### Results: gRPC

- Create account: sent 42 bytes, received 34 bytes.
- Login: sent 42 bytes, received 34 bytes. 7 runs, 1,000 loops each: 1.27 ms ± 43.8 μs
- Send message: sent 175 bytes, received 174 bytes. 7 runs, 1000 loops each: 7.09 ms ± 2.53 ms

We see that our encoding was more verbose which makes sense considering that we included additional checksums/headers which are possibly unneccessary as TCP/IP already does some checkings, and because we had to treat the byte strings of password hashes and tokens as strings to also be compatible with JSON. As expected then, gRPC uses less bandwidth.

Somewhat surprising is that the sending of messages is so slow; this could be due to the different sending/receiving logic we had to implement to allow for the immediate push notifications from the server, even though the sender does not need to wait for the notification to have been sent to the other user.

### Conclusion
**Overall**: After some initial orientation, gRPC was relatively easy to use. VBS is not the biggest fan of the interface–because it works across many different platforms, it doesn't feel very Pythonic. However, the push notification system was a bit of a pain (like in the first exercise), and maybe we haven't fully understood yet what the most native way to implement it using gRPC would be. Could also be an artefact of the way in which Google uses gRPC.

**Difficulty**: Overall, it makes the application a little simpler since we no longer have to worry about encoding, sending, receiving, and parsing requests. gRPC does all of this for us. We were able to remove a few files relating to our wire protocol and the handling of sockets. But of course you give away some flexibility to gRPC which we may have wanted at times.

**Size**: The data being sent around is minimal which is to be expected from a strongly typed protocol. The difference to our own wire protocol are quite stark since we decided to use a very flexible wire protocol which does not require any typing at all (a rather Pythonic approach to the exercise if you will). If bandwidth is an issue, then we should definitely consider using either gRPC or at least modify our wire protocol to have static field names and types.

**Structure**: The structure of the client did not change all that much, we mostly just had to replace the construction of requests sent to the server. The thread responsible for receiving push notifications is a little bit different but not too much. We were able to get rid of the central socket handler which took care of assigning responses to the requests or handling push notifications. The server also largely remained the same, we had to do some trickery to keep our framework with different response types specific to each operation. Especially the stream response was a bit annoying to code, we had to create a fake generator class basically in case anything goes wrong. Not ideal.

**Testing**: Since our previous tests were mostly about the correctness of our wire protocol and the requests we were sending around, those tests are now superfluous–we're trusting the gRPC test suite. We should therefore probably implement API function and/or UI testing. The latter seems really annoying but the former could be helpful, especially if we use mock databases etc.

**UPDATE on Testing**: We decided to implement some more tests, both for the client and for the server. In particular, we're testing the server's actions by using a mock DB, manually creating the gRPC requests and then just skipping the step of actually sending the requests, rather inputting them directly. This now tests our API pretty extensively. The tests aren't really all that specific to gRPC, the only thing we had to take care of was to create the correct request classes for each of the tests. We decided to use the actual gRPC request classes but we could have also created dummy copies of them; this way just seemed to introduce less stuff that we would need to adjust manually if we ever change the gRPC protocol.